In [119]:
import pandas as pd

df = pd.read_csv('lapd-videos.csv')

In [120]:
import re
from utils.list_files import list_files

def extract_number_in_brackets(text):
    pattern = r'\[(\d+)\]'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None

id_to_fn = {}
for fn in list_files('videos'):
    id = extract_number_in_brackets(fn)
    id_to_fn[id] = fn

In [118]:
from utils.imutil import imshow, imread, imwrite
from utils.ffmpeg import vidread, vidreadmeta

def get_year_month(dt):
    return '-'.join(dt.split('-')[:2])
    
to_excerpt = {
    '2023-04': [1],
    '2023-03': [1,4.3],
    '2023-01': [1,7.4],
    '2022-11': [1,7],
    '2022-09': [5,7.7],
    '2022-08': [4.7,7.1],
    '2022-07': [1,7.0],
    '2022-06': [6.1,9.3],
    '2022-05': [7.6,9.4]
}

# this part proved a little less useful than expected

# mapping = {
#     '2023-04': {'class':'10-22', 'date':'2023-04-07'},
#     '2023-03': {'class':'8B-22', 'date':'2023-02-24'},
#     '2023-01': {'class':'7-22', 'date':'2022-12-16'},
#     '2022-11': {'class':'5-22', 'date':'2022-10-21'},
#     '2022-09': {'class':'3-22', 'date':'2022-08-26'},
#     '2022-08': {'class':'2-22', 'date':'2022,07-29'},
#     '2022-07': {'class':'12-21 or 1-22', 'date':'2022-07-01'},
#     '2022-06': {'class':'11-21','date':'2022-06-03'},
#     '2022-05': {'class':'10-21','date':'2022-05-06'}
# }
# def get_graduation_date(year_month):
#     return mapping[year_month]['date']

# i used this, but also downloaded the thumbnails separately
# use_thumb = ['2023-04']

for name,thumb,url,dt in df[['name','thumbnail','link','created_time']].values:
    if name != name:
        continue
    id = url.split('/')[-1]
    if id not in id_to_fn:
        continue
    year_month = get_year_month(dt)
    
    # if year_month in use_thumb:
    #     thumb_fn = 'i.vimeocdn.com/video/' + thumb[29:-6] + '.jpg'
    #     img = imread(thumb_fn)
    #     output_fn = f'images/{name}-{year_month}-{id}-thumb.jpg'
    #     imwrite(output_fn, img)
    #     continue
        
    fn = id_to_fn[id]

    framerate = vidreadmeta(fn)['framerate']

    target_i = [int(e * framerate) for e in to_excerpt[year_month]]
    total = 0
    for i,e in enumerate(vidread(fn)):
        if i in target_i:
            idx = target_i.index(i)
            output_fn = f'images/{name}-{year_month}-{id}-{idx}.jpg'
            imwrite(output_fn, e)
            total += 1
        if total == len(target_i):
            break

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack